## Random Forest: Predicting Terminations

**The aim of this analysis is to see how good we can predict the terminations of a Employee Attrition dataset with a Random Forest model using sklearn.**

Data: https://www.kaggle.com/HRAnalyticRepository/employee-attrition-data (11/29/2020)

In [68]:
import numpy as np
np.random.seed(42)
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import __version__ as skv
import sys

print(f'python: {sys.version.split(" ")[0]}')
print(f'numpy: {np.__version__}')
print(f'pandas: {pd.__version__}')
print(f'sklearn: {skv}')

python: 3.8.3
numpy: 1.18.5
pandas: 1.0.5
sklearn: 0.23.1


In [69]:
df = pd.DataFrame(pd.read_csv('MFG10YearTerminationData.csv'))

#### Get to know the data

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49653 entries, 0 to 49652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   EmployeeID           49653 non-null  int64 
 1   recorddate_key       49653 non-null  object
 2   birthdate_key        49653 non-null  object
 3   orighiredate_key     49653 non-null  object
 4   terminationdate_key  49653 non-null  object
 5   age                  49653 non-null  int64 
 6   length_of_service    49653 non-null  int64 
 7   city_name            49653 non-null  object
 8   department_name      49653 non-null  object
 9   job_title            49653 non-null  object
 10  store_name           49653 non-null  int64 
 11  gender_short         49653 non-null  object
 12  gender_full          49653 non-null  object
 13  termreason_desc      49653 non-null  object
 14  termtype_desc        49653 non-null  object
 15  STATUS_YEAR          49653 non-null  int64 
 16  STAT

In [71]:
df.describe(include=['O'])

,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,city_name,department_name,job_title,gender_short,gender_full,termreason_desc,termtype_desc,STATUS,BUSINESS_UNIT
count,49653,49653,49653,49653,49653,49653,49653,49653,49653,49653,49653,49653,49653
unique,130,5342,4415,1055,40,21,47,2,2,4,3,2,2
top,12/31/2013 0:00,4/27/1956,2/22/1995,1/1/1900,Vancouver,Meats,Meat Cutter,F,Female,Not Applicable,Not Applicable,ACTIVE,STORES
freq,5215,40,50,42450,11211,10269,9984,25898,25898,48168,48168,48168,49068


**There are 40 different cities, 21 different departments and 47 different job titles, that could be grouped.**

In [87]:
df.head()

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE


In [101]:
# Average years of service by job titles and status

df[['length_of_service', 'job_title', 'STATUS']].groupby(['job_title', 'STATUS']).mean().unstack('STATUS')

length_of_service           
STATUS                                     ACTIVE TERMINATED
job_title                                                   
Accounting Clerk                        20.000000  25.000000
Accounts Payable Clerk                  19.545455  22.333333
Accounts Receiveable Clerk              19.230769  21.500000
Auditor                                 18.823529  21.000000
Baker                                    9.930418   9.681319
Bakery Manager                          19.109848  22.095238
Benefits Admin                          19.000000  22.000000
CEO                                     21.500000        NaN
CHief Information Officer               21.500000        NaN
Cashier                                  5.574147   3.294355
Compensation Analyst                    18.823529  20.666667
Corporate Lawyer                        19.428571  20.666667
Customer Service Manager                18.955479  22.571429
Dairy Manager                           19.500000  24.000000
Dairy Person                             7.430238  11.052632
Director, Accounting                    20.500000  24.000000
Director, Accounts Payable              20.500000  24.000000
Director, Accounts Receivable           20.500000  24.000000
Director, Audit                         18.000000  19.000000
Director, Compensation                  18.000000  19.000000
Director, Employee Records              20.500000  24.000000
Director, HR Technology                 20.500000  24.000000
Director, Investments                   18.000000  19.000000
Director, Labor Relations               18.000000  19.000000
Director, Recruitment                   21.500000        NaN
Director, Training                      18.000000  19.000000
Exec Assistant, Finance                 21.500000        NaN
Exec Assistant, Human Resources         21.500000        NaN
Exec Assistant, Legal Counsel           21.500000        NaN
Exec Assistant, VP Stores               21.500000        NaN
HRIS Analyst                            19.106383  21.875000
Investment Analyst                      18.823529  21.666667
Labor Relations Analyst                 18.600000  21.000000
Legal Counsel                           21.500000        NaN
Meat Cutter                             15.525857  13.514124
Meats Manager                           19.603053  22.304348
Processed Foods Manager                 18.798535  21.500000
Produce Clerk                           12.482226  10.762048
Produce Manager                         19.346304  21.904762
Recruiter                               20.222222  22.750000
Shelf Stocker                            6.055696   5.567568
Store Manager                           19.616667  22.258065
Systems Analyst                         18.000000  19.000000
Trainer                                 19.818182  21.500000
VP Finance                              21.500000        NaN
VP Human Resources                      21.500000        NaN
VP Stores                               21.500000        NaN

<br><br>
## Model without grouping any data

In [32]:
# Contert df.STATUS into numerical and separate it as it is the target:

df.STATUS = df.STATUS.map({'ACTIVE':1, 'TERMINATED':0})
target = df.STATUS
df = df.drop('STATUS', axis=1)

In [33]:
# Drop the columns that we do not need for the model:

un_col = ['EmployeeID', 'recorddate_key', 'birthdate_key', 'orighiredate_key',
             'terminationdate_key', 'termreason_desc',
             'termtype_desc', 'STATUS_YEAR', 'store_name', 'gender_full']

df = df.drop(un_col, axis=1)

In [34]:
df.head()

,age,length_of_service,city_name,department_name,job_title,gender_short,BUSINESS_UNIT
0,52,17,Vancouver,Executive,CEO,M,HEADOFFICE
1,53,18,Vancouver,Executive,CEO,M,HEADOFFICE
2,54,19,Vancouver,Executive,CEO,M,HEADOFFICE
3,55,20,Vancouver,Executive,CEO,M,HEADOFFICE
4,56,21,Vancouver,Executive,CEO,M,HEADOFFICE


In [35]:
# Create dummies for the categorial data

dummy_cols = ['city_name', 'department_name', 'job_title', 'gender_short', 'BUSINESS_UNIT']
df = pd.get_dummies(df, columns=dummy_cols)

In [36]:
df.head()

,age,length_of_service,city_name_Abbotsford,city_name_Aldergrove,city_name_Bella Bella,city_name_Blue River,city_name_Burnaby,city_name_Chilliwack,city_name_Cortes Island,city_name_Cranbrook,...,job_title_Store Manager,job_title_Systems Analyst,job_title_Trainer,job_title_VP Finance,job_title_VP Human Resources,job_title_VP Stores,gender_short_F,gender_short_M,BUSINESS_UNIT_HEADOFFICE,BUSINESS_UNIT_STORES
0,52,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,53,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,54,19,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,55,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,56,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [41]:
# Prepare the model:

x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.3, random_state=10)

In [45]:
# Test different parameters

params = {
    'n_estimators': [10, 20, 40, 80, 160],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 4, 8, 10]    
}

In [51]:
clf = RandomForestClassifier()
grid_cv = GridSearchCV(clf, params, cv = 10, n_jobs = -1)
grid_cv.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 4, 8, 10],
                         'n_estimators': [10, 20, 40, 80, 160]})

In [52]:
print(f'Parameter of best model: {grid_cv.best_params_}')
print(f'Score of best model: {grid_cv.best_score_}')

Parameter of best model: {'criterion': 'entropy', 'max_depth': None, 'n_estimators': 80}
Score of best model: 0.9852979526620361
